In [ ]:
"""Jupyter notebook demo for pymatviz widgets."""
# /// script
# dependencies = [
#     "pymatgen>=2024.1.1",
#     "ase>=3.22.0",
#     "phonopy>=2.20.0",
# ]
# ///

# %%
import itertools
import os
from typing import Final

from ase.build import bulk, molecule
from ipywidgets import GridBox, Layout
from phonopy.structure.atoms import PhonopyAtoms
from pymatgen.core import Composition, Lattice, Structure

import pymatviz as pmv


# Test Structure Widget
struct = Structure(
    lattice=Lattice.cubic(3), species=("Fe", "Fe"), coords=((0, 0, 0), (0.5, 0.5, 0.5))
)

structure_widget = pmv.StructureWidget(structure=struct)
structure_widget

StructureWidget(structure={'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattic…

In [ ]:
# %% Test pymatgen Structure MIME type recognition (should render as StructureWidget)
struct

Structure Summary
Lattice
    abc : 3.0 3.0 3.0
 angles : 90.0 90.0 90.0
 volume : 27.0
      A : np.float64(3.0) np.float64(0.0) np.float64(0.0)
      B : np.float64(0.0) np.float64(3.0) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(3.0)
    pbc : True True True
PeriodicSite: Fe (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Fe (1.5, 1.5, 1.5) [0.5, 0.5, 0.5]

In [ ]:
# %% Test Trajectory Widget with simple trajectory of expanding lattice

trajectory = []
for idx in range(5):
    scale = 3.0 + idx * 0.1
    struct_frame = Structure(
        lattice=Lattice.cubic(scale),
        species=("Fe", "Fe"),
        coords=((0, 0, 0), (0.5, 0.5, 0.5)),
    )
    trajectory.append(struct_frame)

trajectory_widget = pmv.TrajectoryWidget(trajectory=trajectory)
trajectory_widget

TrajectoryWidget(trajectory={'frames': [{'structure': {'@module': 'pymatgen.core.structure', '@class': 'Struct…

In [ ]:
# %% Test ASE Atoms MIME type display

ase_atoms = bulk("Al", "fcc", a=4.05)
ase_atoms *= (2, 2, 2)  # Create a 2x2x2 supercell
ase_atoms

Atoms(symbols='Al8', pbc=True, cell=[[0.0, 4.05, 4.05], [4.05, 0.0, 4.05], [4.05, 4.05, 0.0]])

In [ ]:
# %% Test ASE molecule MIME type display

ase_molecule = molecule("H2O")
ase_molecule.center(vacuum=3.0)
ase_molecule

Atoms(symbols='OH2', pbc=False, cell=[6.0, 7.526478, 6.596309])

In [ ]:
# %% Test phonopy atoms MIME type display

lattice = [[4, 0, 0], [0, 4, 0], [0, 0, 4]]
positions = [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]
symbols = ["Na", "Cl"]

phonopy_atoms = PhonopyAtoms(symbols=symbols, positions=positions, cell=lattice)
phonopy_atoms

In [ ]:
# %% Render local flame HDF5 trajectory file

matterviz_traj_dir_url: Final = (
    "https://github.com/janosh/matterviz/raw/33aa595dc/src/site/trajectories"
)
file_name = "flame-gold-cluster-55-atoms.h5"

if not os.path.isfile(f"tmp/{file_name}"):
    import urllib.request

    os.makedirs("tmp", exist_ok=True)

    urllib.request.urlretrieve(  # noqa: S310
        f"{matterviz_traj_dir_url}/{file_name}", f"tmp/{file_name}"
    )

torch_sim_widget = pmv.TrajectoryWidget(
    data_url=f"tmp/{file_name}",
    display_mode="structure+scatter",
    show_force_vectors=False,
)
torch_sim_widget

HTTPError: HTTP Error 404: Not Found

In [ ]:
# %% Render remote ASE trajectory file

githack_traj_dir_url: Final = (
    "https://raw.githack.com/janosh/matterviz/33aa595dc/src/site/trajectories"
)
file_name = "Cr0.25Fe0.25Co0.25Ni0.25-mace-omat-qha.xyz.gz"
ase_traj_widget = pmv.TrajectoryWidget(
    data_url=f"{githack_traj_dir_url}/{file_name}",
    display_mode="structure+scatter",
    show_force_vectors=True,
    force_vector_scale=0.5,
    force_vector_color="#ff4444",
    show_bonds=True,
    bonding_strategy="nearest_neighbor",
    style="height: 600px;",
)
display(ase_traj_widget)  # type: ignore[unresolved-reference]

TrajectoryWidget(data_url='https://raw.githack.com/janosh/matterviz/33aa595dc/src/site/trajectories/Cr0.25Fe0.…

In [ ]:
# %% Render remote flame HDF5 trajectory file

torch_sim_widget = pmv.TrajectoryWidget(
    data_url=f"{githack_traj_dir_url}/flame-gold-cluster-55-atoms.h5",
    display_mode="structure+scatter",
    show_force_vectors=False,
    style="height: 600px;",
)

torch_sim_widget

TrajectoryWidget(data_url='https://raw.githack.com/janosh/matterviz/33aa595dc/src/site/trajectories/torch-sim-…

In [ ]:
# %% Test Composition Widget

comps = (
    "Fe2 O3",
    Composition("Li P O4"),
    dict(Co=20, Cr=20, Fe=20, Mn=20, Ni=20),
    dict(Ti=20, Zr=20, Nb=20, Mo=20, V=20),
)
modes = ("pie", "bar", "bubble")
size = 100
children = [
    pmv.CompositionWidget(
        composition=comp,
        mode=mode,
        style=f"width: {(1 + (mode == 'bar')) * size}px; height: {size}px;",
    )
    for comp, mode in itertools.product(comps, modes)
]
layout = Layout(
    grid_template_columns=f"repeat({len(modes)}, auto)",
    grid_gap="2em 4em",
    padding="2em",
)
GridBox(children=children, layout=layout)

GridBox(children=(CompositionWidget(composition={'Fe': 2.0, 'O': 3.0}, style='width: 100px; height: 100px;'), …